In [ ]:
import pandas as pd
import numpy as np

GSR1_spot = 65  # Pivotal position, it may be necessary to update this if there are changes in the library capacity

def import_data(name):
    # Imports raw data, deletes unused columns, and sets proper column names
    lib = pd.read_csv(name)
    lib = lib.iloc[1:, 4:8].join(lib.iloc[:, 19:])
    lib = lib[lib['Finished'] != "False"]
    lib.columns = ['Weekday or weekend?', 'WeekendTime'] + [str(lib.iloc[0, i])[-11:] for i in range(7, GSR1_spot - 1)]
    lib.columns = lib.columns.tolist() + ['GSR1', 'GSR2', 'GSR3'] + [str(lib.iloc[0, i])[-11:] for i in range(GSR1_spot + 3, lib.shape[1] - 6)]
    lib = lib[list(range(52)) + list(range(56, 61)) + list(range(52, 56) + list(range(61, lib.shape[1])))]
    lib.columns = range(1, len(lib.columns) + 1)
    lib.columns = lib.columns.astype(str)
    return lib

def specific_month(lib, year, month):
    # Creates a data frame for a specific month (string number)
    date = f"{year}-{month}"
    lib_head = lib.loc[0, :]
    lib = lib[lib['4'].str[:7] == date]
    lib = pd.concat([lib_head, lib], ignore_index=True)
    return lib

def weekend_data(lib):
    # Filters for weekend data
    lib_head = lib.loc[0, :]
    lib2 = lib[lib['5'] == "Weekend"]
    lib3 = pd.concat([lib_head, lib2], ignore_index=True)
    return lib3

def weekday_data(lib):
    # Filters for weekday data
    lib_head = lib.loc[0, :]
    lib2 = lib[lib['5'] == "Weekday"]
    lib3 = pd.concat([lib_head, lib2], ignore_index=True)
    return lib3

def monthfinder(num):
    # Returns the character version of the month number
    month_dict = {
        "01": "January",
        "02": "February",
        "03": "March",
        "04": "April",
        "05": "May",
        "06": "June",
        "07": "July",
        "08": "August",
        "09": "September",
        "10": "October",
        "11": "November",
        "12": "December"
    }
    return month_dict.get(num, "Something is wrong_monthfinder")

def find_double_entry(lib):
    # Finds double entries and prints their row and column number
    found = False
    a = lib['6']  # weekend check
    prev = ""
    for i in range(len(a)):
        if not pd.isna(a[i]) and not pd.isna(prev) and a[i] == prev:
            print("Duplicate found!")
            print("Row number:", i)
            print(lib.loc[i, '4'][:20])
            print(lib.loc[i, '6'][:20])
            print("")
            found = True
        prev = a[i]

    b = lib.iloc[:, -1]  # weekday check
    prev = ""
    for i in range(len(b)):
        if not pd.isna(b[i]) and not pd.isna(prev) and b[i] == prev:
            print("Duplicate found!")
            print("Row number:", i)
            print(lib.loc[i, '4'][:20])
            print(lib.iloc[i, -1][:20])
            print("")
            found = True
        prev = b[i]

    if not found:
        print("No duplicates found!")

def numcheck(data):
    # Checks for NA or invalid input
    def select(e1, e2):
        return e2 if pd.isna(e1) else e1

    data = data.iloc[1:, :]
    no_mistake = True
    for j in range(data.shape[0]):
        for i in [GSR1_spot, GSR1_spot + 1, GSR1_spot + 2, data.shape[1] - 5, data.shape[1] - 4, data.shape[1] - 3, data.shape[1] - 2, data.shape[1] - 1]:
            if pd.isna(pd.to_numeric(data.iloc[j, i])):
                no_mistake = False
                print(f"Row number: {j + 1} Column Number: {i} ({data.iloc[0, i]})")
                print(f"Date: {data.iloc[j, 3]} Shift: {select(data.iloc[j, 5], data.iloc[j, -1])}")
                print(f"Entered: {data.iloc[j, i]}\n-")

    if no_mistake:
        print("All inputs are valid!")

def capacitycheck(data, showplace=False):
    # Checks for capacity limits
    def capacity(place):
        if place == GSR1_spot:
            return 4
        elif place == GSR1_spot + 1:
            return 4
        elif place == GSR1_spot + 2:
            return 4
        elif place == data.shape[1] - 5:
            return 5
        elif place == data.shape[1] - 4:
            return 5
        elif place == data.shape[1] - 3:
            return 7
        elif place == data.shape[1] - 2:
            return 7
        elif place == data.shape[1] - 1:
            return 26
        else:
            print("Inputs are wrong!")

    def capacity_covid(place):
        if place == GSR1_spot:
            return 2
        elif place == GSR1_spot + 1:
            return 2
        elif place == GSR1_spot + 2:
            return 2
        elif place == data.shape[1] - 5:
            return 2
        elif place == data.shape[1] - 4:
            return 2
        elif place == data.shape[1] - 3:
            return 3
        elif place == data.shape[1] - 2:
            return 3
        elif place == data.shape[1] - 1:
            return 26
        else:
            print("Inputs are wrong!")

    def select(e1, e2):
        return e2 if pd.isna(e1) else e1

    data = data.iloc[1:, :]
    no_over = True
    for j in range(data.shape[0]):
        for i in [GSR1_spot, GSR1_spot + 1, GSR1_spot + 2, data.shape[1] - 5, data.shape[1] - 4, data.shape[1] - 3, data.shape[1] - 2, data.shape[1] - 1]:
            if not pd.isna(pd.to_numeric(data.iloc[j, i])):
                amount = pd.to_numeric(data.iloc[j, i])
                cap = capacity_covid(i)
                if amount > cap:
                    no_over = False
                    if showplace:
                        print(f"Row number: {j + 1} Column Number: {i}")
                    print(f"({data.iloc[0, i]})")
                    print(f"Date: {data.iloc[j, 3]} Shift: {select(data.iloc[j, 5], data.iloc[j, -1])}")
                    print(f"Capacity: {cap} Entered: {amount}\n-")

    if no_over:
        print("No over capacity detected!")


In [ ]:
def findmissing(input):
    weekend = ["WEEKEND - 11am", "WEEKEND - 12pm", "WEEKEND - 1pm", "WEEKEND - 2pm", "WEEKEND - 3pm",
               "WEEKEND - 4pm", "WEEKEND - 5pm", "WEEKEND - 6pm"]
    weekday = ["WEEKDAY - 10am", "WEEKDAY - 12pm", "WEEKDAY - 2pm", "WEEKDAY - 4pm", "WEEKDAY - 6pm",
               "WEEKDAY - 7pm", "WEEKDAY - 8pm", "WEEKDAY - 9pm", "WEEKDAY - 10pm"]

    dataset = input.iloc[1:, :]
    days = []
    a = 1
    expected = 0
    missing = 0
    weekend_missing = 0

    for item in dataset['4']:
        day = item[8:10]
        if day in days:
            continue
        else:
            days.append(day)
            a += 1
            sublib = dataset[dataset['4'].str[8:10] == day]
            entry_type = sublib.iloc[0, 4]

            if entry_type == "Weekday":
                expected += 9
                if sublib.shape[0] == 9:
                    pass
                else:
                    subday = []
                    b = 1
                    subsub = sublib.iloc[:, -1]
                    for shift in subsub:
                        subday.append(shift)
                        b += 1
                    print(day)
                    diff = list(set(weekday) - set(subday))
                    print(diff)
                    missing += len(diff)
            else:  # if weekend
                expected += 8
                if sublib.shape[0] == 8:
                    pass
                else:
                    subday = []
                    b = 1
                    for shift in sublib['6']:
                        subday.append(shift)
                        b += 1
                    print(day)
                    diff = list(set(weekend) - set(subday))
                    print(diff)
                    missing += len(diff)
                    weekend_missing += len(diff)

    print("The expected number of entries is:", expected)
    print("The actual number of entries is:", (expected - missing))
    print("The number of missing entries is:", missing)
    percentage_missing = (missing / expected) * 100
    print("The percentage of missing entries is:", percentage_missing)
    print("The number of missing entries on the weekend is:", weekend_missing)


In [ ]:
def count_aggregate(lib):
    # For each space, counts the amount of like/dislike/neutral
    # Easy to remove/add new sections - you may have to edit import data though
    # First row (column row) is filled with dummy variables which will be replaced by actual names
    initial_ncol = lib.shape[1]

    # Main Stack (MS)
    temp = pd.DataFrame({
        "MS_Like": [0] * len(lib),
        "MS_Dislike": [0] * len(lib),
        "MS_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = 7
    capacity = 16
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Individual Seat Level 1 (IS_L1)
    temp = pd.DataFrame({
        "IS_L1_Like": [0] * len(lib),
        "IS_L1_Dislike": [0] * len(lib),
        "IS_L1_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 6
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Shared Seat Level 1 (SS_L1)
    temp = pd.DataFrame({
        "SS_L1_Like": [0] * len(lib),
        "SS_L1_Dislike": [0] * len(lib),
        "SS_L1_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 32
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Soft Seat Left Level 1 (SoftL_L1)
    temp = pd.DataFrame({
        "SoftL_L1_Like": [0] * len(lib),
        "SoftL_L1_Dislike": [0] * len(lib),
        "SoftL_L1_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 4
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Shared Seat Level 2 (SS_L2)
    temp = pd.DataFrame({
        "SS_L2_Like": [0] * len(lib),
        "SS_L2_Dislike": [0] * len(lib),
        "SS_L2_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1 + 3  # GSR1-3 entries are in between, thus +3 added
    capacity = 18
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Soft Individual Seat Level 2 (SoftIS_L2)
    temp = pd.DataFrame({
        "SoftIS_L2_Like": [0] * len(lib),
        "SoftIS_L2_Dislike": [0] * len(lib),
        "SoftIS_L2_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 2
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Soft Shared Seat Level 2 (SoftSS_L2)
    temp = pd.DataFrame({
        "SoftSS_L2_Like": [0] * len(lib),
        "SoftSS_L2_Dislike": [0] * len(lib),
        "SoftSS_L2_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 8
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Individual Seat 24H (IS_24H)
    temp = pd.DataFrame({
        "IS_24H_Like": [0] * len(lib),
        "IS_24H_Dislike": [0] * len(lib),
        "IS_24H_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 4
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Soft Individual Seat 24H (SoftIS_24H)
    temp = pd.DataFrame({
        "SoftIS_24H_Like": [0] * len(lib),
        "SoftIS_24H_Dislike": [0] * len(lib),
        "SoftIS_24H_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 4
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Soft Shared Seat 24H (SoftSS_24H)
    temp = pd.DataFrame({
        "SoftSS_24H_Like": [0] * len(lib),
        "SoftSS_24H_Dislike": [0] * len(lib),
        "SoftSS_24H_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 4
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Shared Seat 24H (SS_24H)
    temp = pd.DataFrame({
        "SS_24H_Like": [0] * len(lib),
        "SS_24H_Dislike": [0] * len(lib),
        "SS_24H_Neutral": [0] * len(lib)
    })

    lib = pd.concat([lib, temp], axis=1)

    startpos = endpos + 1
    capacity = 42
    endpos = startpos + capacity - 1

    for i in range(len(lib)):
        for j in range(startpos, endpos + 1):
            if lib.iloc[i, j] == "Neutral":
                lib.iloc[i, -1] = pd.to_numeric(lib.iloc[i, -1]) + 1
            elif lib.iloc[i, j] == "Dislike":
                lib.iloc[i, -2] = pd.to_numeric(lib.iloc[i, -2]) + 1
            else:
                lib.iloc[i, -3] = pd.to_numeric(lib.iloc[i, -3]) + 1

    # Remove unnecessary columns
    lib = lib.drop(lib.columns[7: (GSR1_spot - 1) + (GSR1_spot + 3) : (initial_ncol - 6)], axis=1)

    # Rename columns with appropriate names
    names = lib.columns[15:]
    lib.iloc[0, 15:] = names
    lib.columns = list(range(1, len(lib.columns) + 1))

    return lib


In [ ]:

def convert_previous(lib):
    # Stacks Total Occupancy
    lib['Stacks_Total'] = pd.to_numeric(lib.iloc[1:, 15]) + pd.to_numeric(lib.iloc[1:, 16])

    # Stacks Dislike
    lib['Stacks_DL'] = pd.to_numeric(lib.iloc[1:, 16])

    # Level 1 Total Occupancy
    level1_total = (
        pd.to_numeric(lib.iloc[1:, 18]) + pd.to_numeric(lib.iloc[1:, 19]) +
        pd.to_numeric(lib.iloc[1:, 21]) + pd.to_numeric(lib.iloc[1:, 22]) +
        pd.to_numeric(lib.iloc[1:, 24]) + pd.to_numeric(lib.iloc[1:, 25])
    )
    lib['Level1_Total'] = level1_total

    # Level 1 Dislike
    level1_dislike = pd.to_numeric(lib.iloc[1:, 19]) + pd.to_numeric(lib.iloc[1:, 22]) + pd.to_numeric(lib.iloc[1:, 25])
    lib['Level1_DL'] = level1_dislike

    # Level 2 Total Occupancy
    level2_total = (
        pd.to_numeric(lib.iloc[1:, 27]) + pd.to_numeric(lib.iloc[1:, 28]) +
        pd.to_numeric(lib.iloc[1:, 30]) + pd.to_numeric(lib.iloc[1:, 31]) +
        pd.to_numeric(lib.iloc[1:, 33]) + pd.to_numeric(lib.iloc[1:, 34])
    )
    lib['Level2_Total'] = level2_total

    # Level 2 Dislike
    level2_dislike = pd.to_numeric(lib.iloc[1:, 31]) + pd.to_numeric(lib.iloc[1:, 34]) + pd.to_numeric(lib.iloc[1:, 28])
    lib['Level2_DL'] = level2_dislike

    # 24 H Total Occupancy
    total_24h = (
        pd.to_numeric(lib.iloc[1:, 39]) + pd.to_numeric(lib.iloc[1:, 40]) +
        pd.to_numeric(lib.iloc[1:, 42]) + pd.to_numeric(lib.iloc[1:, 43]) +
        pd.to_numeric(lib.iloc[1:, 45]) + pd.to_numeric(lib.iloc[1:, 46]) +
        pd.to_numeric(lib.iloc[1:, 36]) + pd.to_numeric(lib.iloc[1:, 37])
    )
    lib['24H_Total'] = total_24h

    # 24 H Dislike
    dislike_24h = (
        pd.to_numeric(lib.iloc[1:, 40]) + pd.to_numeric(lib.iloc[1:, 43]) +
        pd.to_numeric(lib.iloc[1:, 46]) + pd.to_numeric(lib.iloc[1:, 37])
    )
    lib['24H_DL'] = dislike_24h

    # Reorder columns
    col_order = [1, 2, 3, 4, 5, 6, 15, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23,
                 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
                 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
    lib = lib[col_order]
    lib.columns = range(1, len(lib.columns) + 1)
    
    return lib

def addcolumn(lib):
    lib2 = lib.copy()

    for i in range(1, len(lib2)):
        # Creating GSR13 column
        item = [pd.to_numeric(lib2.iloc[i, j]) for j in range(7, 10)]
        if pd.isna(item).any():
            lib2.at[i, 23] = None
        else:
            lib2.at[i, 23] = sum(item)

        # Creating GSR45 column
        item = [pd.to_numeric(lib2.iloc[i, j]) for j in range(10, 12)]
        if pd.isna(item).any():
            lib2.at[i, 24] = None
        else:
            lib2.at[i, 24] = sum(item)

        # Creating GSR67 column
        item = [pd.to_numeric(lib2.iloc[i, j]) for j in range(12, 14)]
        if pd.isna(item).any():
            lib2.at[i, 25] = None
        else:
            lib2.at[i, 25] = sum(item)

        # Creating Total column
        item = [pd.to_numeric(lib2.iloc[i, j]) for j in range(7, 16)] + \
               [pd.to_numeric(lib2.iloc[i, 18])] + \
               [pd.to_numeric(lib2.iloc[i, 20])] + \
               [pd.to_numeric(lib2.iloc[i, 22])]
        if pd.isna(item).any():
            lib2.at[i, 26] = None
        else:
            lib2.at[i, 26] = sum(item)

        # Creating Total without GSR/CL column
        item = [pd.to_numeric(lib2.iloc[i, 16)] + pd.to_numeric(lib2.iloc[i, 18]) + pd.to_numeric(lib2.iloc[i, 20]) + pd.to_numeric(lib2.iloc[i, 22])]
        if pd.isna(item).any():
            lib2.at[i, 27] = None
        else:
            lib2.at[i, 27] = sum(item)

    libhead = ["GSR1-3", "GSR4-5", "GSR6-7", "Total", "Total_NO GSR/CL"]
    lib2.iloc[0, 23:28] = libhead
    lib2.columns = range(1, len(lib2.columns) + 1)

    return lib2

In [ ]:
import pandas as pd

def datafinder(mon, section, time):
    def weekday_data(mon):
        return mon

    def weekend_data(mon):
        return mon

    hours = []

    if time == "weekday":
        hours = ["WEEKDAY - 10am", "WEEKDAY - 12pm", "WEEKDAY - 2pm", "WEEKDAY - 4pm",
                 "WEEKDAY - 6pm", "WEEKDAY - 7pm", "WEEKDAY - 8pm", "WEEKDAY - 9pm", "WEEKDAY - 10pm"]
        mon.columns.values[6] = "hour"
        mon = weekday_data(mon)
    elif time == "weekend":
        hours = ["WEEKEND - 11am", "WEEKEND - 12pm", "WEEKEND - 1pm", "WEEKEND - 2pm",
                 "WEEKEND - 3pm", "WEEKEND - 4pm", "WEEKEND - 5pm", "WEEKEND - 6pm"]
        mon.columns.values[5] = "hour"
        mon = weekend_data(mon)
    else:
        print("Wrong time input datafinder")
        return None

    if section == "All":
        mon.columns.values[27] = "this"
    elif section == "4main":
        mon.columns.values[28] = "this"
    elif section == "L1":
        mon.columns.values[18] = "this"
    elif section == "Stacks":
        mon.columns.values[16] = "this"
    elif section == "GSR13":
        mon.columns.values[24] = "this"
    elif section == "GSR45":
        mon.columns.values[25] = "this"
    elif section == "GSR67":
        mon.columns.values[26] = "this"
    elif section == "L2":
        mon.columns.values[20] = "this"
    elif section == "CLab":
        mon.columns.values[15] = "this"
    elif section == "24H":
        mon.columns.values[22] = "this"
    else:
        print("Wrong section input datafinder")
        return None

    result = []

    for shift in hours:
        subdata = mon[mon["hour"] == shift]
        a = pd.to_numeric(subdata["this"])
        s = a.sum(na_rm=True)
        len_ = len(a[~pd.isna(a)])
        avg = s / len_
        result.append(avg)

    return result

In [ ]:
# ALANI START